In [1]:
import numpy as np

In [2]:
%load_ext autoreload
%autoreload 2

In [3]:
world.reset()

NameError: name 'world' is not defined

In [4]:
import getpass
user = getpass.getuser()

In [5]:
from omni.isaac.kit import SimulationApp

usd_path = f"omniverse://localhost/Users/{user}/test_hand.usd"


simulation_app = SimulationApp({"headless": True, "open_usd": usd_path, "livesync_usd": usd_path})


Starting kit application with the fillowing args:  ['/home/yizhou/.local/share/ov/pkg/isaac_sim-2022.1.0/exts/omni.isaac.kit/omni/isaac/kit/simulation_app.py', '/home/yizhou/.local/share/ov/pkg/isaac_sim-2022.1.0/apps/omni.isaac.sim.python.kit', '--/app/tokens/exe-path=/home/yizhou/.local/share/ov/pkg/isaac_sim-2022.1.0/kit', '--/persistent/app/viewport/displayOptions=3094', '--/rtx/materialDb/syncLoads=True', '--/rtx/hydra/materialSyncLoads=True--/omni.kit.plugin/syncUsdLoads=True', '--/app/renderer/resolution/width=1280', '--/app/renderer/resolution/height=720', '--/app/window/width=1440', '--/app/window/height=900', '--/renderer/multiGpu/enabled=True', '--ext-folder', '/home/yizhou/.local/share/ov/pkg/isaac_sim-2022.1.0/exts', '--ext-folder', '/home/yizhou/.local/share/ov/pkg/isaac_sim-2022.1.0/apps', '--portable', '--no-window']
Passing the following args to the base kit application:  ['-f', '/home/yizhou/.local/share/jupyter/runtime/kernel-384664b0-193a-4f92-9111-0f39e99d52c0.json

OmniAssetFileFormat


[1.703s] [ext: omni.mdl-0.1.0] startup
[1.723s] [ext: omni.kit.menu.create-1.0.2] startup
[1.724s] [ext: omni.kit.window.file_exporter-1.0.4] startup
[1.725s] [ext: omni.kit.window.drop_support-1.0.0] startup
[1.725s] [ext: omni.kit.material.library-1.3.10] startup
[1.729s] [ext: omni.kit.window.property-1.6.3] startup
[1.730s] [ext: omni.kit.context_menu-1.3.9] startup
[1.733s] [ext: omni.kit.window.file_importer-1.0.4] startup
[1.734s] [ext: omni.kit.stage_templates-1.1.2] startup
[1.736s] [ext: omni.kit.widget.stage-2.6.15] startup
[1.740s] [ext: omni.kit.window.file-1.3.16] startup
[1.743s] [ext: omni.debugdraw-0.1.0] startup
[1.749s] [ext: omni.kit.window.content_browser-2.4.28] startup
[1.759s] [ext: omni.kit.widget.prompt-1.0.1] startup
[1.760s] [ext: omni.kit.property.usd-3.14.8] startup
[1.798s] [ext: omni.hydra.engine.stats-1.0.0] startup
[1.804s] [ext: omni.kit.widget.settings-1.0.0] startup
[1.805s] [ext: omni.graph.tools-1.3.5] startup
[1.870s] [ext: omni.graph.core-2.27.0

2022-08-10 00:47:59 [2,017ms] [Error] [rtx.neuraylib.plugin] [DYNLIB:IO]   0.1   DYNLIB io   error: /home/yizhou/.local/share/ov/pkg/isaac_sim-2022.1.0/kit/python/bin/libs/iray/libnvindex.so: cannot open shared object file: No such file or directory
2022-08-10 00:47:59 [2,017ms] [Error] [rtx.neuraylib.plugin] [INDEX:MAIN]   0.1   INDEX  main error: Failed to load /home/yizhou/.local/share/ov/pkg/isaac_sim-2022.1.0/kit/python/bin/libs/iray/libnvindex.so


Warp initialized:
   Version: 0.2.1
   Using CUDA device: NVIDIA GeForce RTX 3090
   Using CPU compiler: /usr/bin/g++
[2.748s] [ext: omni.kit.window.preferences-1.2.1] startup
[2.788s] [ext: omni.kit.ui_test-1.2.0] startup
[2.790s] [ext: omni.graph.ui-1.6.1] startup
[2.814s] [ext: omni.kvdb-0.0.0] startup
[2.818s] [ext: omni.kit.widget.searchfield-1.0.6] startup
[2.819s] [ext: omni.convexdecomposition-1.4.12] startup
[2.823s] [ext: omni.graph.action-1.17.0] startup
[2.834s] [ext: omni.localcache-0.0.0] startup
[2.837s] [ext: omni.usdphysics-1.4.12] startup
[2.840s] [ext: omni.graph.scriptnode-0.5.0] startup
[2.843s] [ext: omni.physx-1.4.12-5.1] startup
2022-08-10 00:47:59 [2,850ms] [Warning] [omni.kvdb.plugin] wasn't able to load the meta database, trying to repair it ...
2022-08-10 00:47:59 [2,940ms] [Warning] [omni.kvdb.plugin] repair failed
[2.948s] [ext: omni.kit.usd_undo-0.1.0] startup
[2.950s] [ext: omni.graph.nodes-1.25.0] startup
[2.968s] [ext: omni.physx.commands-1.4.12-5.1] s

In [6]:
from omni.isaac.core import World
world = World()

In [7]:
from hand_env import HandEnv

# env = HandEnv("/World/Hand/Bones/l_carpal_mid", "/World/Hand*/Bones/l_thumbSkeleton_grp/l_distalThumb_mid")
env = HandEnv("/World/Hand/Bones/l_carpal_mid", "/World/AnchorXform")

In [8]:
world.reset()

In [9]:
env.start()
world.scene.add(env.robots)

simlation context numpy None
num_envs 1


In [10]:
world.reset()

In [11]:
hand_pos, hand_rot = env.robots.get_world_poses()

In [12]:
hand_pos, hand_rot

(array([[-1.0132790e-06,  3.5930425e-06, -1.3814410e-03]], dtype=float32),
 array([[9.9999291e-01, 3.2006856e-03, 1.9685545e-03, 2.5742559e-04]],
       dtype=float32))

In [13]:
env.xforms.set_world_poses(positions=np.array([[0,0,0]]), orientations = np.array([[1, 0, 0, 0]])) # WXYZ

for _ in range(30):
    world.step(render=True)

# DOF examine

In [20]:
for i, path in enumerate(env.robots._physics_view.dof_paths[0]):
    print(i, path)

0 /World/Hand/Bones/l_thumbSkeleton_grp/l_metacarpalThumb_mid/Thumb_metacarpal_D6DriverJoint
1 /World/Hand/Bones/l_thumbSkeleton_grp/l_metacarpalThumb_mid/Thumb_metacarpal_D6DriverJoint
2 /World/Hand/Bones/l_indexSkeleton_grp/l_proximalIndex_mid/Index_proximal_D6DriverJoint
3 /World/Hand/Bones/l_indexSkeleton_grp/l_proximalIndex_mid/Index_proximal_D6DriverJoint
4 /World/Hand/Bones/l_middleSkeleton_grp/l_proximalMiddle_mid/Middle_proximal_D6DriverJoint
5 /World/Hand/Bones/l_middleSkeleton_grp/l_proximalMiddle_mid/Middle_proximal_D6DriverJoint
6 /World/Hand/Bones/l_pinkySkeleton_grp/l_proximalPinky_mid/Pinky_proximal_D6DriverJoint
7 /World/Hand/Bones/l_pinkySkeleton_grp/l_proximalPinky_mid/Pinky_proximal_D6DriverJoint
8 /World/Hand/Bones/l_ringSkeleton_grp/l_proximalRing_mid/Ring_proximal_D6DriverJoint
9 /World/Hand/Bones/l_ringSkeleton_grp/l_proximalRing_mid/Ring_proximal_D6DriverJoint
10 /World/Hand/Bones/l_thumbSkeleton_grp/l_proximalThumb_mid/Thumb_proximal_RevoluteJoint
11 /World/Ha

In [51]:
for i in range(100):
    i  = i / 5
    pos = np.array([
        [ i * 0.03,  i * 0.04, 
          i * 0.01,  -i * 0.04,  
          i * 0.005, -i * 0.04, 
         -i * 0.02, -i * 0.04,  
         -i * 0.01, -i * 0.04,  
         -i * 0.02,  -i * 0.03,  -i * 0.03,  -i * 0.03,  -i * 0.03,
         -i * 0.02,  -i * 0.03,  -i * 0.03,  -i * 0.03,  -i * 0.03, 
        ],
    ])

    # pos = np.random.randn(2,25)
    env.robots.set_joint_position_targets(pos) # 
    world.step(render=True)

# Stage one

In [22]:
i = 30
pos = np.array([
        [ 0,  0, 
          i * 0.02,  0,  
          0, 0, 
         -i * 0.02, 0,  
         -i * 0.01, 0,  
          0, 0, 0, 0, 0, # 0,  -i * 0.03,  -i * 0.03,  -i * 0.03,  -i * 0.03,
          0, 0, 0, 0, 0, # 0,  -i * 0.03,  -i * 0.03,  -i * 0.03,  -i * 0.03, 
        ],
    ])

for _ in range(100):
    env.robots.set_joint_position_targets(pos) 
    world.step(render=True)

# Stage two

In [18]:
i = 30
pos = np.array([
        [ 0,  0, 
          i * 0.01,  0,  
          0, 0, 
         -i * 0.02, 0,  
         -i * 0.01, 0,  
          0,  -i * 0.03,  -i * 0.03,  -i * 0.03,  -i * 0.03,
          0,  -i * 0.03,  -i * 0.03,  -i * 0.03,  -i * 0.03, 
        ],
    ])

for _ in range(100):
    env.robots.set_joint_position_targets(pos) 
    env.robots.set_world_poses(hand_pos,hand_rot)
    world.step(render=True)

# Stage three

In [ ]:
for _ in range(100):
    env.robots.set_joint_position_targets(pos) 
    
    hand_pos, hand_rot = env.robots.get_world_poses()
    hand_pos[...,0] += 0.1
    env.robots.set_world_poses(hand_pos, hand_rot)
    
    world.step(render=True)

------------------

In [ ]:
env.robots.get_joint_positions()

In [ ]:
joint_v = np.array([[
     1,  0,  
     0., 0.,
     0., 0., 
     0., 0., 
     0., 0., 
     0, 0, 0, 0, 0,
     0, 0, 0, 0, 0
]]).transpose()

In [ ]:
joint_v.shape

In [ ]:
jacobians = env.robots._physics_view.get_jacobians()

In [ ]:
jacobians.shape

In [ ]:
delta_pos_rot = np.matmul(jacobians[0], joint_v)
for i in range(len(delta_pos_rot)):
    print(i, delta_pos_rot[i].transpose())

# Jacobian

In [ ]:
world.reset()
world.step(render=True)

In [ ]:
thumb_pos, thumb_rot = env.xforms.get_world_poses()

In [ ]:
thumb_rot = thumb_rot[:,[1,2,3,0]] # WXYZ -> XYZW

In [ ]:
thumb_pos, thumb_rot

In [ ]:
target_pos = thumb_pos.copy()#np.array([[-0.37809137,  -0.3650283  , 0.05804292]])

In [ ]:
target_pos[...,1] -= 0.5

In [ ]:
target_pos

In [ ]:
env.move_finger_to_fast(target_pos, thumb_rot, world, finger = "thumb")

In [ ]:
target_pos = np.zeros((env.num_envs, 20)) 
# target_pos = np.random.randn(env.num_envs, 20) 

In [ ]:
env.num_envs

In [ ]:
target_pos

In [ ]:
for _ in range(100):
    world.step(render=True)
    thumb_pos, thumb_rot = env.xforms.get_world_poses()
    env.robots.set_joint_position_targets(target_pos) 
    print("thumb_pos", thumb_pos)

In [ ]:
env.robots.get_joint_positions()

# Control

In [ ]:
pos, rot = env.robots.get_local_poses()

In [ ]:
pos, rot

In [ ]:
# pos[...,0] = -10

In [ ]:
env.robots.set_local_poses(pos,rot)
world.step(render=True)

In [ ]:
hand_prim = world.scene.stage.GetPrimAtPath("/World/Hand")

In [ ]:
from pxr import UsdGeom, Gf

In [ ]:
mat = Gf.Matrix4f().SetTranslate(Gf.Vec3f(0, 0, 0))

In [ ]:
import omni

In [ ]:
omni.kit.commands.execute(
    "TransformPrimCommand",
    path="/World/Hand",
    new_transform_matrix=mat,
)

In [52]:
world.reset()